# Sync with External Sensors

Many experimental setups record data from multiple sensors in parallel. This data needs to be synced temporally for a joint analysis. All eye tracking data you record with Pupil Invisible is accurately timestamped, which makes this easily possible.

In this guide you will learn how to sync eye tracking data from Pupil Invisible to any other timestamped external sensor using the `pd.merge_asof` function of Pandas. As an example, we will sync a heart rate sensor with a Pupil Invisible recording to produce a gaze overlay visualization with real-time heart rate of a man jogging.

TODO
- What heart rate sensor die we use?

## Dependencies of this Guide
To be written.
- Required Python libs
    - pandas av fitdecode tqdm opencv-python
- Where to download the code
- Where to download the example data
- How to put data into data folder

## Loading all Data
For the example visualization we need the scene video and gaze data from the Pupil Invisible recording, and the heart rate data from the [??]() recording.

The heart rate data can be read using the `fitdecode` module. For more details check out the implementation of the [load_fit_data]() implementation.

The gaze data CSV file can be read using Pandas. All we need from it is the timestamps and the gaze values.

For the scene video, we initially only need its timestamps and the corresponding frame indices for matching, we don't have to touch the actual video frames yet.

In [1]:
import pandas as pd

from decode_fit import load_fit_data


fit_path = "data/eye-tracking-run.FIT"
fit = load_fit_data(fit_path)

gaze_path = "data/demo-recording/running_rd-4a40d94d/gaze.csv"
gaze = pd.read_csv(gaze_path)
gaze["timestamp [ns]"] = pd.to_datetime(gaze["timestamp [ns]"])
gaze = gaze[["timestamp [ns]", "gaze x [px]", "gaze y [px]"]]

world_ts_path = "data/demo-recording/running_rd-4a40d94d/world_timestamps.csv"
world_ts = pd.read_csv(world_ts_path)
world_ts["frame_index"] = world_ts.index
world_ts["timestamp [ns]"] = pd.to_datetime(world_ts["timestamp [ns]"])

## Timestamp Matching

In [2]:
df = pd.merge_asof(world_ts, fit, left_on="timestamp [ns]", right_on="timestamp", direction="nearest")
df = pd.merge_asof(df, gaze, left_on="timestamp [ns]", right_on="timestamp [ns]", direction="nearest")
df.head()

,section id,recording id,timestamp [ns],frame_index,latitude,longitude,altitude,timestamp,heart_rate,cadence,speed,gaze x [px],gaze y [px]
0,3e2512bf-4389-42f4-86af-20feebe937e8,4a40d94d-7fbd-439b-be98-7ac7cf2c13c2,2022-07-09 17:19:08.694,0,56.140768,10.187518,NaN,2022-07-09 17:20:19,95,0,0.0,464.678,747.029
1,3e2512bf-4389-42f4-86af-20feebe937e8,4a40d94d-7fbd-439b-be98-7ac7cf2c13c2,2022-07-09 17:19:08.744,1,56.140768,10.187518,NaN,2022-07-09 17:20:19,95,0,0.0,464.678,747.029
2,3e2512bf-4389-42f4-86af-20feebe937e8,4a40d94d-7fbd-439b-be98-7ac7cf2c13c2,2022-07-09 17:19:08.794,2,56.140768,10.187518,NaN,2022-07-09 17:20:19,95,0,0.0,464.678,747.029
3,3e2512bf-4389-42f4-86af-20feebe937e8,4a40d94d-7fbd-439b-be98-7ac7cf2c13c2,2022-07-09 17:19:08.844,3,56.140768,10.187518,NaN,2022-07-09 17:20:19,95,0,0.0,464.678,747.029
4,3e2512bf-4389-42f4-86af-20feebe937e8,4a40d94d-7fbd-439b-be98-7ac7cf2c13c2,2022-07-09 17:19:08.894,4,56.140768,10.187518,NaN,2022-07-09 17:20:19,95,0,0.0,464.678,747.029


In [ ]:
import av

cut_off_index = 15000
world_vid_path = "/Users/marc/Downloads/raw-data-export (2)/running_rd-4a40d94d/3e2512bf_0.0-2504.94.mp4"
world_lookup = {}
for index, packet in enumerate(av.open(world_vid_path).demux(video=0)):
    world_lookup[index] = packet
    
    if index == cut_off_index:
        break

world_ts = world_ts.iloc[:cut_off_index]

vis_vid_path = "/Users/marc/Downloads/raw-data-export (2)/running_rd-4a40d94d/vis.mp4"

In [64]:
import cv2

for idx, row in df.iterrows():
    if idx < 30 * 60 * 10:
        continue
    
    frame = world_lookup[row["frame_index"]].decode()
    try:
        frame = frame[0]
    except IndexError:
        continue

    img = frame.to_ndarray(format="bgr24")
    
    gaze = (int(row["gaze x [px]"]), int(row["gaze y [px]"]))
    cv2.circle(img, gaze, 50, (0, 0, 255), 5)
    
    heart_rate = row["heart_rate"]
    cv2.putText(img, f"HR: {heart_rate}", (50,100), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,0,0), 3)

    cv2.imshow("Scene Video + Gaze + Heartrate", img)
    key = cv2.waitKey(1)

: 

In [3]:
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt
import numpy as np


def visualize(img, df, idx):
    row = df.iloc[idx]
    gaze = (int(row["gaze x [px]"]), int(row["gaze y [px]"]))
    cv2.circle(img, gaze, 50, (0, 0, 255), 5)
    
    heart_rate = row["heart_rate"]
    cv2.putText(img, f"HR: {heart_rate}", (50,100), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,0,0), 3)

    # Make a random plot...
    fig = plt.figure()
    # ax = fig.add_subplot(111)

    pad = 50
    heart_rate = df.iloc[idx - pad: idx + pad].heart_rate

    plt.plot(np.arange(len(heart_rate)) + idx - pad, heart_rate, color="blue")
    plt.xlim(idx - pad, idx + pad)

    # If we haven't already shown or saved the plot, then we need to
    # draw the figure first...
    fig.canvas.draw()

    # Now we can save it to a numpy array.
    data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    w_v, h_v, _ = data.shape
    w, h, _ = img.shape

    img[w-w_v:w, 0:h_v, :] = data

    return img

original_container = av.open(str(world_vid_path))
original_video_stream = original_container.streams.video[0]

visualization_container = av.open(str(vis_vid_path), "w")

try:
    visualization_video = visualization_container.add_stream("h264_nvenc")
except Exception as e:
    print("nvenc not available", e)
    visualization_video = visualization_container.add_stream("h264")

visualization_video.options["bf"] = "0"
visualization_video.options["movflags"] = "faststart"
visualization_video.gop_size = original_video_stream.gop_size
visualization_video.codec_context.height = original_video_stream.height
visualization_video.codec_context.width = original_video_stream.width
visualization_video.codec_context.time_base = original_video_stream.time_base
visualization_video.codec_context.bit_rate = original_video_stream.bit_rate

progress = tqdm(unit=" frames", total=len(df))
with visualization_container:
    for idx, row in df.iterrows():
        frame = world_lookup[row["frame_index"]].decode()
        try:
            frame = frame[0]
        except IndexError:
            continue

        img = frame.to_ndarray(format="bgr24")
        vis_img = visualize(img, df, idx)

        # cv2.imshow("Scene Video + Gaze + Heartrate", vis_img)
        # cv2.waitKey(1)
        
        new_frame = frame.from_ndarray(vis_img, format="bgr24")
        new_frame.pts = frame.pts
        new_frame.time_base = original_video_stream.time_base
        packets = visualization_video.encode(new_frame)
        progress.update()
        visualization_container.mux(packets)
    # encode and mux frames that have been queued internally by the encoders
    visualization_container.mux(visualization_video.encode())

nvenc not available h264_nvenc


  0%|          | 19/15000 [00:00<11:08, 22.40 frames/s] /var/folders/k3/8x0h0s854dq0jc83qmhmzn3w0000gn/T/ipykernel_3690/4244152212.py:16: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()
  5%|▍         | 743/15000 [00:46<14:50, 16.00 frames/s]